In [51]:
import os 
import sys
import importlib
project_root = os.path.abspath("../")
sys.path.append(project_root)
from src.data import process_data
importlib.reload(process_data)
from src.data.load_data import load_data
from src.data.process_data import preproces ,clean_and_tokenize

In [52]:
preprocessed_data = preproces(load_data(tmdb_5000_movies='../data/raw/tmdb_5000_movies.csv', tmdb_5000_credits='../data/raw/tmdb_5000_credits.csv'))

In [53]:
X = preprocessed_data[['overview_title','cast', 'crew']]
y = preprocessed_data['genres']

In [54]:
genres_list =[genre for genres in y for genre in genres]
element_to_index = {}
idx = 0
for e in genres_list:
    if e in element_to_index:
        continue
    element_to_index[e] = idx
    idx += 1

print(element_to_index)


{'Action': 0, 'Adventure': 1, 'Fantasy': 2, 'Science Fiction': 3, 'Crime': 4, 'Drama': 5, 'Thriller': 6, 'Animation': 7, 'Family': 8, 'Western': 9, 'Comedy': 10, 'Romance': 11, 'Horror': 12, 'Mystery': 13, 'History': 14, 'War': 15, 'Music': 16, 'Documentary': 17, 'Foreign': 18, 'TV Movie': 19}


In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train['overview_title'][0])

In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Avatar


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer_cast = TfidfVectorizer(max_features=2500, stop_words='english')
tfidf_vectorizer_crew = TfidfVectorizer(max_features=2500, stop_words='english')
cast_tf_idf = tfidf_vectorizer_cast.fit_transform(X_train['cast'].apply(lambda x : ' '.join(x)))
crew_tf_idf = tfidf_vectorizer_crew.fit_transform(X_train['crew'].apply(lambda x : ' '.join(x)))

In [57]:
cast_tf_idf_test = tfidf_vectorizer_cast.transform(X_test['cast'].apply(lambda x : ' '.join(x)))
crew_tf_idf_test = tfidf_vectorizer_crew.transform(X_test['crew'].apply(lambda x : ' '.join(x)))

In [58]:
import numpy as np
def doc2vec(tokens,word2vec):
    vectors = [word2vec.wv[token] for token in tokens if token in word2vec.wv]
    vector = np.mean(vectors, axis = 0)
    return vector

In [59]:
print(type(X_train['overview_title'][0]))
importlib.reload(process_data)

<class 'str'>


<module 'src.data.process_data' from 'e:\\nlp_projects\\Movie Genre Classification with Multi-label Output\\src\\data\\process_data.py'>

In [60]:
from gensim.models import Word2Vec 
X_train['overview_title'] = X_train['overview_title'].apply(clean_and_tokenize)
X_test['overview_title'] = X_test['overview_title'].apply(clean_and_tokenize)
print(X_train['overview_title'])

4687    [fidgety, wisecracking, video, store, clerk, d...
2951    [greenwich, village, early, gifted, volatile, ...
4071    [agent, back, second, installment, james, bond...
4579    [king, arthur, accompanied, squire, recruits, ...
2197    [ally, darling, anna, faris, realizing, shes, ...
                              ...                        
4426    [fine, step, uplifting, family, drama, centeri...
466     [hoping, alter, events, past, th, century, inv...
3092    [seemingly, perfect, family, moves, suburban, ...
3772    [drying, west, coast, alcoholic, hit, man, bef...
860     [burlesque, lounge, best, days, behind, tess, ...
Name: overview_title, Length: 3842, dtype: object


In [61]:
word2vec = Word2Vec(sentences=X_train['overview_title'],vector_size=300,window=5,min_count=2)
train_overview_title_vec = [doc2vec(sentence,word2vec) for sentence in X_train['overview_title']]
test_overview_title_vec = [doc2vec(sentence,word2vec) for sentence in X_test['overview_title']]

In [62]:
from sklearn.preprocessing import normalize
train_overview_title_vec = normalize(train_overview_title_vec)
test_overview_title_vec = normalize(test_overview_title_vec)

In [63]:
from scipy.sparse import csr_matrix,hstack

train_overview_title_vec = csr_matrix(train_overview_title_vec)
test_overview_title_vec = csr_matrix(test_overview_title_vec)

In [64]:
print(train_overview_title_vec.shape)
print(cast_tf_idf.shape)
print(crew_tf_idf.shape)

(3842, 300)
(3842, 2500)
(3842, 2500)


In [65]:
X_train_sparse = hstack([train_overview_title_vec, cast_tf_idf, crew_tf_idf])
X_test_sparse = hstack([test_overview_title_vec, cast_tf_idf_test, crew_tf_idf_test])

# Then convert to dense
X_train = X_train_sparse.toarray()
X_test = X_test_sparse.toarray()

In [66]:
print(X_train.shape)

(3842, 5300)


In [67]:
# from torch.utils.data import Dataset,DataLoader 
# import torch
def encode_genres(genres,element_to_index):
    encoded = [0]* len(element_to_index)
    for genre in genres:
        encoded[element_to_index[genre]] = 1 
    
    return encoded

# class movies_dataset(Dataset):
#     def __init__(self,X,y,element_to_index):
#         self.X = X 
#         self.y = y 
#         self.encoded = [encode_genres(genres,element_to_index) for genres in y]
    
#     def __len__(self):
#         return len(self.X)
    
#     def __getitem__(self,idx):
#         return torch.tensor(self.X[idx],dtype=torch.float32),torch.tensor(self.encoded[idx],dtype=torch.float32)

# train_dataset = movies_dataset(X_train,y_train,element_to_index)
# train_loader = DataLoader(train_dataset,batch_size= 32,shuffle=True)
# test_dataset = movies_dataset(X_test,y_test,element_to_index)
# test_loader = DataLoader(test_dataset,batch_size=32,shuffle=True)

In [68]:
# from models import FFN 
# importlib.reload(FFN)
# from models.FFN import FNN_model
# fnn_model = FNN_model(input_dim=4300,hidden_dim=300,output_dim=20)

In [69]:
# from models.train_and_evaluate import train_model
# criterian = torch.nn.BCELoss()
# optim = torch.optim.SGD(fnn_model.parameters())
# train_model(fnn_model,criterian,optim=optim,epochs=250,loader=train_loader)

In [70]:
# from models import train_and_evaluate 
# importlib.reload(train_and_evaluate)
# from models.train_and_evaluate import evaluate 
# print(evaluate(fnn_model,test_loader))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
encoded_train = [encode_genres(genres,element_to_index) for genres in y_train]

In [77]:
model = OneVsRestClassifier(LogisticRegression())
model.fit(X_train,encoded_train)
preds = model.predict_proba(X_test)
preds = (preds >= 0.2).astype(int)

In [72]:
from sklearn.metrics import f1_score
encoded_test = [encode_genres(genres,element_to_index) for genres in y_test]
f1_micro = f1_score(encoded_test, preds, average='micro')

In [73]:
print(f1_micro)

0.49596309111880044


In [74]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.multiclass import OneVsRestClassifier

# model = OneVsRestClassifier(GradientBoostingClassifier())


In [75]:
# model.fit(X_train,encoded_train)
# preds = model.predict(X_test)

In [76]:
# encoded_test = [encode_genres(genres,element_to_index) for genres in y_test]
# f1_micro = f1_score(encoded_test, preds, average='micro')
# print(f1_micro)